In [2]:
import pandas as pd
import sys
from datetime import datetime

class MedicalComplaintsProcessor:
    def __init__(self, file_path):
        """
        Инициализация процессора для работы с файлом медицинских обращений
        
        :param file_path: путь к файлу Excel
        """
        self.file_path = file_path
        self.sheets_data = {}
        self.days_column_name = None  # Для хранения имени используемого столбца с днями
        
    def load_data(self):
        """
        Загрузка данных из всех листов Excel файла с обработкой ошибок
        """
        try:
            # Чтение всех листов из файла
            with pd.ExcelFile(self.file_path) as xls:
                for sheet_name in xls.sheet_names:
                    # Пропускаем скрытые листы
                    if 'hidden' in sheet_name.lower():
                        continue
                        
                    # Читаем данные с листа
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    self.sheets_data[sheet_name] = self._clean_data(df)
                    
            return self
        except ImportError as e:
            print(f"Ошибка: {e}\nПожалуйста, установите openpyxl: pip install openpyxl")
            sys.exit(1)
        except Exception as e:
            print(f"Произошла ошибка при загрузке файла: {e}")
            sys.exit(1)
    
    def _clean_data(self, df):
        """
        Очистка данных: удаление пустых строк и столбцов, нормализация заголовков
        
        :param df: DataFrame для очистки
        :return: Очищенный DataFrame
        """
        # Удаляем полностью пустые строки
        df = df.dropna(how='all')
        
        # Удаляем полностью пустые столбцы
        df = df.dropna(axis=1, how='all')
        
        # Нормализуем названия столбцов
        df.columns = [self._normalize_column_name(col) for col in df.columns]
        
        return df
    
    def _normalize_column_name(self, col_name):
        """
        Нормализация названия столбца
        
        :param col_name: исходное название столбца
        :return: нормализованное название
        """
        if not isinstance(col_name, str):
            return col_name
            
        return (col_name.strip()
                .replace('(Не изменять)', '')
                .replace('(', '')
                .replace(')', '')
                .strip()
                .replace('  ', ' '))
    
    def get_main_data(self):
        """
        Получение основных данных из листа с обращениями
        
        :return: DataFrame с основными данными
        """
        # Ищем лист с основными данными
        for sheet_name, df in self.sheets_data.items():
            if any(keyword in sheet_name.lower() 
                   for keyword in ['мед.жалобы', 'мед. блок', 'основные', 'main']):
                return df
        
        # Если не нашли, возвращаем первый непустой лист
        return next(iter(self.sheets_data.values()))
    
    def get_report_data(self):
        """
        Получение данных из листа с отчетом
        
        :return: DataFrame с отчетными данными или None, если лист не найден
        """
        for sheet_name, df in self.sheets_data.items():
            if any(keyword in sheet_name.lower() 
                   for keyword in ['отчет', 'report', 'статистика']):
                return df
        return None
    
    def get_complaints_by_department(self, department_name):
        """
        Получение обращений по подразделению
        
        :param department_name: название подразделения (можно часть названия)
        :return: DataFrame с отфильтрованными обращениями
        """
        main_data = self.get_main_data()
        
        # Возможные названия столбца с подразделениями
        department_columns = [
            'Виновное Подразделение Клиника',
            'Виновное подразделение',
            'Подразделение',
            'Клиника',
            'Отделение',
            'Department',
            'BU'
        ]
        
        # Находим правильный столбец
        dept_col = self._find_column(main_data, department_columns)
                
        if dept_col is None:
            available = "\n".join(main_data.columns)
            raise ValueError(
                f"Не найден столбец с подразделениями. Доступные столбцы:\n{available}"
            )
            
        # Фильтруем по подразделению (регистронезависимо)
        filtered = main_data[
            main_data[dept_col].str.contains(department_name, na=False, case=False)
        ]
        
        return filtered
    
    def get_complaints_by_status(self, status):
        """
        Получение обращений по статусу
        
        :param status: статус обращения (можно часть статуса)
        :return: DataFrame с отфильтрованными обращениями
        """
        main_data = self.get_main_data()
        
        # Возможные названия столбца со статусами
        status_columns = [
            'Status Reason',
            'Статус',
            'Status',
            'Состояние',
            'Текущий статус'
        ]
        
        # Находим правильный столбец
        status_col = self._find_column(main_data, status_columns)
                
        if status_col is None:
            available = "\n".join(main_data.columns)
            raise ValueError(
                f"Не найден столбец со статусами. Доступные столбцы:\n{available}"
            )
            
        # Фильтруем по статусу (регистронезависимо)
        filtered = main_data[
            main_data[status_col].str.contains(status, na=False, case=False)
        ]
        
        return filtered
    
    def get_overdue_complaints(self, days_threshold=30):
        """
        Получение просроченных обращений
        
        :param days_threshold: порог количества дней для определения просрочки
        :return: DataFrame с просроченными обращениями
        """
        main_data = self.get_main_data()
        
        # Пытаемся найти столбец с количеством дней
        days_col = self._find_days_column(main_data)
        
        # Конвертируем в числовой формат, если необходимо
        if not pd.api.types.is_numeric_dtype(main_data[days_col]):
            main_data[days_col] = pd.to_numeric(main_data[days_col], errors='coerce')
            
        # Фильтруем по порогу, исключая NaN значения
        overdue = main_data[
            (main_data[days_col] > days_threshold) & 
            (main_data[days_col].notna())
        ].copy()
        
        # Сохраняем имя используемого столбца для последующего использования
        self.days_column_name = days_col
        
        return overdue
    
    def _find_days_column(self, df):
        """
        Поиск столбца с количеством дней ответа
        
        :param df: DataFrame для поиска
        :return: имя найденного столбца
        """
        # Возможные названия столбца с днями
        days_columns = [
            'Количество дней ответа ответственным исполнителем',
            'Days to response',
            'Срок ответа',
            'Количество дней',
            'Дни ответа',
            'Ответ за дней',
            'Срок исполнения дни',
            'Время ответа'
        ]
        
        # Сначала ищем прямой столбец с днями
        days_col = self._find_column(df, days_columns)
        
        # Если не нашли, пытаемся вычислить по датам
        if days_col is None:
            days_col = self._calculate_days_column(df)
            
        return days_col
    
    def _calculate_days_column(self, df):
        """
        Вычисление дней ответа по разнице дат
        
        :param df: DataFrame для вычисления
        :return: имя созданного столбца с днями
        """
        # Ищем столбцы с датами
        date_sent_col = self._find_column(df, [
            'Дата отправки',
            'Date sent',
            'Дата отправки в Виновное подразделение',
            'Sent date'
        ])
        
        date_received_col = self._find_column(df, [
            'Дата получения ответа',
            'Date received',
            'Дата получения ответа от Виновного подразделения',
            'Received date'
        ])
        
        if not date_sent_col or not date_received_col:
            available = "\n".join(df.columns)
            raise ValueError(
                "Не найден столбец с количеством дней ответа и невозможно вычислить по датам.\n"
                f"Доступные столбцы:\n{available}"
            )
        
        # Конвертируем в datetime если нужно
        for col in [date_sent_col, date_received_col]:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Вычисляем разницу в днях
        df['Вычисленные дни ответа'] = (df[date_received_col] - df[date_sent_col]).dt.days
        
        return 'Вычисленные дни ответа'
    
    def _find_column(self, df, possible_names):
        """
        Поиск столбца по возможным названиям
        
        :param df: DataFrame для поиска
        :param possible_names: список возможных названий
        :return: имя найденного столбца или None
        """
        for name in possible_names:
            if name in df.columns:
                return name
        return None
    
    def get_summary_stats(self):
        """
        Получение сводной статистики по обращениям
        
        :return: словарь с статистикой
        """
        main_data = self.get_main_data()
        stats = {'total_complaints': len(main_data)}
        
        # Статистика по статусам
        status_col = self._find_column(main_data, [
            'Status Reason', 'Статус', 'Status'
        ])
        if status_col:
            stats['by_status'] = main_data[status_col].value_counts().to_dict()
        
        # Статистика по подразделениям
        dept_col = self._find_column(main_data, [
            'Виновное Подразделение Клиника', 'Подразделение', 'Department'
        ])
        if dept_col:
            stats['by_department'] = main_data[dept_col].value_counts().to_dict()
        
        # Статистика по времени ответа
        try:
            days_col = self.days_column_name or self._find_days_column(main_data)
            if days_col:
                if not pd.api.types.is_numeric_dtype(main_data[days_col]):
                    main_data[days_col] = pd.to_numeric(main_data[days_col], errors='coerce')
                
                stats.update({
                    'avg_response_days': main_data[days_col].mean(),
                    'max_response_days': main_data[days_col].max(),
                    'min_response_days': main_data[days_col].min(),
                    'median_response_days': main_data[days_col].median(),
                    'response_days_column': days_col
                })
        except:
            pass
            
        return stats
    
    def save_to_excel(self, output_path, df=None, sheet_name='Результат'):
        """
        Сохранение результатов в Excel файл
        
        :param output_path: путь для сохранения
        :param df: DataFrame для сохранения (если None, сохраняются все данные)
        :param sheet_name: название листа
        """
        try:
            if df is None:
                with pd.ExcelWriter(output_path) as writer:
                    for sheet_name, df in self.sheets_data.items():
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
            else:
                df.to_excel(output_path, sheet_name=sheet_name, index=False)
            print(f"Данные успешно сохранены в {output_path}")
        except Exception as e:
            print(f"Ошибка при сохранении файла: {e}")


def main():
    try:
        # Указываем путь к файлу
        input_path = r"C:/Users/1/IDE/table1.xlsx"
        output_path = r"C:/Users/1/IDE/результат_анализа.xlsx"
        
        # Инициализация и загрузка данных
        print("Загрузка данных...")
        processor = MedicalComplaintsProcessor(input_path)
        processor.load_data()
        
        # Анализ данных
        print("\nАнализ данных:")
        main_data = processor.get_main_data()
        print(f"Всего обращений: {len(main_data)}")
        
        # Пример фильтрации по подразделению
        department = "КДЦБ"
        dept_complaints = processor.get_complaints_by_department(department)
        print(f"\nОбращений по подразделению '{department}': {len(dept_complaints)}")
        
        # Пример фильтрации по статусу
        status = "Завершено"
        completed = processor.get_complaints_by_status(status)
        print(f"\nОбращений со статусом '{status}': {len(completed)}")
        
        # Просроченные обращения
        overdue = processor.get_overdue_complaints(30)
        print(f"\nПросроченных обращений (>30 дней): {len(overdue)}")
        if len(overdue) > 0:
            print("\nПримеры просроченных обращений:")
            print(overdue.head(3)[['Номер обращения', 'Виновное Подразделение Клиника', 
                                 processor.days_column_name or 'Дни ответа']])
        
        # Сводная статистика
        stats = processor.get_summary_stats()
        print("\nСводная статистика:")
        print(f"Среднее время ответа: {stats.get('avg_response_days', 'N/A'):.1f} дней")
        print(f"Максимальное время ответа: {stats.get('max_response_days', 'N/A')} дней")
        print(f"Минимальное время ответа: {stats.get('min_response_days', 'N/A')} дней")
        
        # Сохранение результатов
        processor.save_to_excel(output_path)
        
    except Exception as e:
        print(f"\nОшибка: {e}")
        print("Убедитесь, что:")
        print("1. Файл существует и доступен по указанному пути")
        print("2. Установлены все зависимости (pip install pandas openpyxl)")
        print("3. Файл не открыт в другой программе")
        print("4. Структура файла соответствует ожидаемой")


if __name__ == "__main__":
    main()

Загрузка данных...

Анализ данных:
Всего обращений: 128

Ошибка: Не найден столбец с подразделениями. Доступные столбцы:
Номер обращения
Status Reason
Ответственный сотрудник ЦПП
Тип обращения
Дата приема в работу
Дата и время приема обращения
Канал поступления
Инициатор
Отношение инициатора к МЕДСИ
Контактный телефон инициатора
Пациент
Дата рождения Пациент Контакт
Контактный телефон пациента
Электронная почта Пациент Контакт
Название компании Пациент Контакт
Категория пациента Пациент Контакт
Суть обращения запись СТРОГО со слов Клиента
Тип претензии
Классификация
ФИО Инициатора
Сотрудник
Дата отправки Ответственному исполнителю
Дата получения ответа от Ответственного исполнителя
Дата согласования
Согласующий сотрудник
Финальный ответ
Обращение обосновано?
new Обращение обосновано?
order Обращение обосновано?
Виновное Подразделение/Клиника
Дата отправки в Виновное подразделение
Срок работы с обращением календарных дней
Срок работы с обращением рабочих дней
Сумма возврата р.
Дата полу

In [1]:
import pandas as pd
import sys
from datetime import datetime

class MedicalComplaintsProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.sheets_data = {}
        self.days_column_name = None

    def load_data(self):
        try:
            with pd.ExcelFile(self.file_path) as xls:
                for sheet_name in xls.sheet_names:
                    if 'hidden' in sheet_name.lower():
                        continue
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    self.sheets_data[sheet_name] = self._clean_data(df)
            return self
        except Exception as e:
            print(f"Ошибка при загрузке файла: {e}")
            sys.exit(1)

    def _clean_data(self, df):
        df = df.dropna(how='all')
        df = df.dropna(axis=1, how='all')
        df.columns = [self._normalize_column_name(col) for col in df.columns]
        return df

    def _normalize_column_name(self, col_name):
        if not isinstance(col_name, str):
            return col_name
        return (col_name.strip()
                .replace('(Не изменять)', '')
                .replace('(', '')
                .replace(')', '')
                .strip()
                .replace('  ', ' '))

    def get_main_data(self):
        for sheet_name, df in self.sheets_data.items():
            if any(keyword in sheet_name.lower() 
                   for keyword in ['мед.жалобы', 'мед. блок', 'основные', 'main']):
                return df
        return next(iter(self.sheets_data.values()))

    def analyze_overdue_complaints(self, threshold_days=30):
        """Анализ просроченных обращений с разбивкой по подразделениям"""
        main_data = self.get_main_data().copy()
        
        # Находим необходимые столбцы
        date_sent_col = self._find_column(main_data, [
            'Дата отправки в Виновное подразделение',
            'Дата отправки',
            'Date sent'
        ])
        
        date_received_col = self._find_column(main_data, [
            'Дата получения ответа от Виновного подразделения',
            'Дата получения ответа',
            'Date received'
        ])
        
        status_col = self._find_column(main_data, ['Status Reason', 'Статус'])
        dept_col = self._find_column(main_data, [
            'Виновное Подразделение/Клиника', 
            'Подразделение',
            'Виновное Подразделение'
        ])
        
        if not date_sent_col or not date_received_col or not dept_col:
            raise ValueError("Не найдены необходимые столбцы в данных")

        # Конвертируем даты
        main_data[date_sent_col] = pd.to_datetime(main_data[date_sent_col], errors='coerce')
        main_data[date_received_col] = pd.to_datetime(main_data[date_received_col], errors='coerce')
        
        # Вычисляем разницу в днях
        main_data['days_to_response'] = (main_data[date_received_col] - main_data[date_sent_col]).dt.days
        
        # Фильтруем только завершенные и отправленные на согласование обращения
        if status_col:
            main_data = main_data[main_data[status_col].isin(['Завершено', 'Отправлено на согласование'])]
        
        # Разделяем на просроченные и непросроченные
        main_data['is_overdue'] = main_data['days_to_response'] > threshold_days
        
        # Общее количество просроченных обращений
        total_overdue = main_data['is_overdue'].sum()
        
        # Группируем по подразделениям
        dept_stats = main_data.groupby(dept_col).agg(
            total_complaints=('days_to_response', 'size'),
            overdue_complaints=('is_overdue', 'sum'),
            avg_days=('days_to_response', 'mean'),
            max_days=('days_to_response', 'max'),
            min_days=('days_to_response', 'min')
        ).reset_index()
        
        # Рассчитываем проценты
        dept_stats['overdue_percent'] = round(
            dept_stats['overdue_complaints'] / dept_stats['total_complaints'] * 100, 1)
        
        # Добавляем долю от общего числа просроченных обращений
        dept_stats['share_of_total_overdue'] = round(
            dept_stats['overdue_complaints'] / total_overdue * 100, 1)
        
        # Сортируем по проценту просрочки
        dept_stats = dept_stats.sort_values('overdue_percent', ascending=False)
        
        # Общая статистика
        total_stats = {
            'total_complaints': len(main_data),
            'overdue_complaints': total_overdue,
            'overdue_percent': round(main_data['is_overdue'].mean() * 100, 1),
            'avg_days': round(main_data['days_to_response'].mean(), 1),
            'max_days': main_data['days_to_response'].max(),
            'min_days': main_data['days_to_response'].min(),
            'threshold_days': threshold_days
        }
        
        return {
            'total_stats': total_stats,
            'department_stats': dept_stats.to_dict('records'),
            'raw_data': main_data
        }

    def print_detailed_stats(self, threshold_days=30):
        """Вывод подробной статистики с разбивкой по подразделениям и долями"""
        analysis = self.analyze_overdue_complaints(threshold_days)
        
        print("\nОбщая статистика по обращениям:")
        print(f"Порог просрочки: {analysis['total_stats']['threshold_days']} дней")
        print(f"Всего обращений: {analysis['total_stats']['total_complaints']}")
        print(f"Просроченных обращений: {analysis['total_stats']['overdue_complaints']} ({analysis['total_stats']['overdue_percent']}%)")
        print(f"Среднее время ответа: {analysis['total_stats']['avg_days']} дней")
        print(f"Максимальное время ответа: {analysis['total_stats']['max_days']} дней")
        print(f"Минимальное время ответа: {analysis['total_stats']['min_days']} дней")
        
        print("\nСтатистика по подразделениям (отсортировано по % просрочки):")
        print("{:<60} {:<10} {:<10} {:<10} {:<15} {:<10} {:<10} {:<10}".format(
            "Подразделение", "Всего", "Просрочено", "% проср", "Доля от общ", "Ср.дни", "Мин", "Макс"
        ))
        
        for dept in analysis['department_stats']:
            print("{:<60} {:<10} {:<10} {:<10} {:<15} {:<10.1f} {:<10} {:<10}".format(
                dept['Виновное Подразделение/Клиника'][:55],
                dept['total_complaints'],
                dept['overdue_complaints'],
                dept['overdue_percent'],
                f"{dept['share_of_total_overdue']}%",
                dept['avg_days'],
                dept['min_days'],
                dept['max_days']
            ))

    def _find_column(self, df, possible_names):
        """Вспомогательный метод для поиска столбца"""
        for name in possible_names:
            if name in df.columns:
                return name
        return None

    def save_analysis_to_excel(self, output_file, threshold_days=30):
        """Сохранение полного анализа в Excel файл"""
        analysis = self.analyze_overdue_complaints(threshold_days)
        
        with pd.ExcelWriter(output_file) as writer:
            # Сохраняем общую статистику
            total_stats_df = pd.DataFrame([analysis['total_stats']])
            total_stats_df.to_excel(writer, sheet_name='Общая статистика', index=False)
            
            # Сохраняем статистику по подразделениям
            dept_stats_df = pd.DataFrame(analysis['department_stats'])
            dept_stats_df.to_excel(writer, sheet_name='По подразделениям', index=False)
            
            # Сохраняем исходные данные с пометкой просрочки
            raw_data = analysis['raw_data']
            raw_data.to_excel(writer, sheet_name='Исходные данные', index=False)
            
            # Добавляем сводную таблицу
            pivot = raw_data.pivot_table(
                index='Виновное Подразделение/Клиника',
                columns='is_overdue',
                values='days_to_response',
                aggfunc='count',
                fill_value=0
            )
            pivot.columns = ['В срок', 'Просрочено']
            pivot['Всего'] = pivot['В срок'] + pivot['Просрочено']
            pivot['% просрочки'] = round(pivot['Просрочено'] / pivot['Всего'] * 100, 1)
            pivot.to_excel(writer, sheet_name='Сводная таблица')

if __name__ == "__main__":
    try:
        # Убедитесь, что файл table1.xlsx находится в той же директории, что и скрипт
        processor = MedicalComplaintsProcessor("table1.xlsx")
        processor.load_data()
        
        # Вывод статистики в консоль
        processor.print_detailed_stats(30)
        
        # Сохранение полного отчета в Excel
        processor.save_analysis_to_excel("medical_complaints_analysis.xlsx")
        
        print("\nАнализ успешно завершен. Результаты сохранены в medical_complaints_analysis.xlsx")
        
    except Exception as e:
        print(f"Ошибка: {e}")
        print("Проверьте путь к файлу и его структуру")


Общая статистика по обращениям:
Порог просрочки: 30 дней
Всего обращений: 101
Просроченных обращений: 0 (0.0%)
Среднее время ответа: -0.0 дней
Максимальное время ответа: 0.0 дней
Минимальное время ответа: -1.0 дней

Статистика по подразделениям (отсортировано по % просрочки):
Подразделение                                                Всего      Просрочено % проср    Доля от общ     Ср.дни     Мин        Макс      
Астрадамский, Азбука Здоровья                                1          0          0.0        nan%            nan        nan        nan       
Благовещенка (вз) (в сети КППн)                              7          0          0.0        nan%            0.0        0.0        0.0       
Брянск                                                       1          0          0.0        nan%            0.0        0.0        0.0       
Бутово +ДА (в сети КППн)                                     1          0          0.0        nan%            0.0        0.0        0.0       
Гамалеи

In [3]:
import pandas as pd
import sys
from datetime import datetime

class DataLoader:
    def __init__(self, file_path):
        self.file_path = file_path
        self.sheets_data = {}
    
    def load_data(self):
        """Загрузка и предварительная обработка данных из Excel"""
        try:
            with pd.ExcelFile(self.file_path) as xls:
                for sheet_name in xls.sheet_names:
                    if 'hidden' in sheet_name.lower():
                        continue
                    df = pd.read_excel(xls, sheet_name=sheet_name)
                    self.sheets_data[sheet_name] = self._clean_data(df)
            return self.sheets_data
        except Exception as e:
            print(f"Ошибка при загрузке файла: {e}")
            sys.exit(1)
    
    def _clean_data(self, df):
        """Очистка и нормализация данных"""
        df = df.dropna(how='all')
        df = df.dropna(axis=1, how='all')
        df.columns = [self._normalize_column_name(col) for col in df.columns]
        return df
    
    def _normalize_column_name(self, col_name):
        """Нормализация названий столбцов"""
        if not isinstance(col_name, str):
            return col_name
        return (col_name.strip()
                .replace('(Не изменять)', '')
                .replace('(', '')
                .replace(')', '')
                .strip()
                .replace('  ', ' '))
    
    def get_main_data(self):
        """Получение основных данных (медицинских жалоб)"""
        for sheet_name, df in self.sheets_data.items():
            if any(keyword in sheet_name.lower() 
                   for keyword in ['мед.жалобы', 'мед. блок', 'основные', 'main']):
                return df
        return next(iter(self.sheets_data.values()))


class ComplaintsAnalyzer:
    def __init__(self, data_loader):
        self.data_loader = data_loader
        self.main_data = None
    
    def prepare_data(self):
        """Подготовка данных для анализа"""
        self.main_data = self.data_loader.get_main_data().copy()
        return self.main_data
    
    def analyze_overdue_complaints(self, threshold_days=30):
        """Анализ просроченных обращений"""
        if self.main_data is None:
            self.prepare_data()
        
        # Находим необходимые столбцы
        date_sent_col = self._find_column([
            'Дата отправки в Виновное подразделение',
            'Дата отправки',
            'Date sent'
        ])
        
        date_received_col = self._find_column([
            'Дата получения ответа от Виновного подразделения',
            'Дата получения ответа',
            'Date received'
        ])
        
        status_col = self._find_column(['Status Reason', 'Статус'])
        dept_col = self._find_column([
            'Виновное Подразделение/Клиника', 
            'Подразделение',
            'Виновное Подразделение'
        ])
        
        if not date_sent_col or not date_received_col or not dept_col:
            raise ValueError("Не найдены необходимые столбцы в данных")

        # Конвертируем даты
        self.main_data[date_sent_col] = pd.to_datetime(self.main_data[date_sent_col], errors='coerce')
        self.main_data[date_received_col] = pd.to_datetime(self.main_data[date_received_col], errors='coerce')
        
        # Вычисляем разницу в днях
        self.main_data['days_to_response'] = (self.main_data[date_received_col] - self.main_data[date_sent_col]).dt.days
        
        # Фильтруем только завершенные и отправленные на согласование обращения
        if status_col:
            self.main_data = self.main_data[self.main_data[status_col].isin(['Завершено', 'Отправлено на согласование'])]
        
        # Разделяем на просроченные и непросроченные
        self.main_data['is_overdue'] = self.main_data['days_to_response'] > threshold_days
        
        # Общее количество просроченных обращений
        total_overdue = self.main_data['is_overdue'].sum()
        
        # Группируем по подразделениям
        dept_stats = self.main_data.groupby(dept_col).agg(
            total_complaints=('days_to_response', 'size'),
            overdue_complaints=('is_overdue', 'sum'),
            avg_days=('days_to_response', 'mean'),
            max_days=('days_to_response', 'max'),
            min_days=('days_to_response', 'min')
        ).reset_index()
        
        # Рассчитываем проценты
        dept_stats['overdue_percent'] = round(
            dept_stats['overdue_complaints'] / dept_stats['total_complaints'] * 100, 1)
        
        # Добавляем долю от общего числа просроченных обращений
        dept_stats['share_of_total_overdue'] = round(
            dept_stats['overdue_complaints'] / total_overdue * 100, 1)
        
        # Сортируем по проценту просрочки
        dept_stats = dept_stats.sort_values('overdue_percent', ascending=False)
        
        # Общая статистика
        total_stats = {
            'total_complaints': len(self.main_data),
            'overdue_complaints': total_overdue,
            'overdue_percent': round(self.main_data['is_overdue'].mean() * 100, 1),
            'avg_days': round(self.main_data['days_to_response'].mean(), 1),
            'max_days': self.main_data['days_to_response'].max(),
            'min_days': self.main_data['days_to_response'].min(),
            'threshold_days': threshold_days
        }
        
        return {
            'total_stats': total_stats,
            'department_stats': dept_stats.to_dict('records'),
            'raw_data': self.main_data
        }
    
    def _find_column(self, possible_names):
        """Поиск столбца по возможным названиям"""
        for name in possible_names:
            if name in self.main_data.columns:
                return name
        return None


class ReportGenerator:
    def __init__(self, analyzer):
        self.analyzer = analyzer
    
    def print_detailed_stats(self, threshold_days=30):
        """Вывод подробной статистики"""
        analysis = self.analyzer.analyze_overdue_complaints(threshold_days)
        
        print("\nОбщая статистика по обращениям:")
        print(f"Порог просрочки: {analysis['total_stats']['threshold_days']} дней")
        print(f"Всего обращений: {analysis['total_stats']['total_complaints']}")
        print(f"Просроченных обращений: {analysis['total_stats']['overdue_complaints']} ({analysis['total_stats']['overdue_percent']}%)")
        print(f"Среднее время ответа: {analysis['total_stats']['avg_days']} дней")
        print(f"Максимальное время ответа: {analysis['total_stats']['max_days']} дней")
        print(f"Минимальное время ответа: {analysis['total_stats']['min_days']} дней")
        
        print("\nСтатистика по подразделениям (отсортировано по % просрочки):")
        print("{:<60} {:<10} {:<10} {:<10} {:<15} {:<10} {:<10} {:<10}".format(
            "Подразделение", "Всего", "Просрочено", "% проср", "Доля от общ", "Ср.дни", "Мин", "Макс"
        ))
        
        for dept in analysis['department_stats']:
            print("{:<60} {:<10} {:<10} {:<10} {:<15} {:<10.1f} {:<10} {:<10}".format(
                dept['Виновное Подразделение/Клиника'][:55],
                dept['total_complaints'],
                dept['overdue_complaints'],
                dept['overdue_percent'],
                f"{dept['share_of_total_overdue']}%",
                dept['avg_days'],
                dept['min_days'],
                dept['max_days']
            ))
    
    def save_to_excel(self, output_file, threshold_days=30):
        """Сохранение отчета в Excel"""
        analysis = self.analyzer.analyze_overdue_complaints(threshold_days)
        
        with pd.ExcelWriter(output_file) as writer:
            # Общая статистика
            total_stats_df = pd.DataFrame([analysis['total_stats']])
            total_stats_df.to_excel(writer, sheet_name='Общая статистика', index=False)
            
            # Статистика по подразделениям
            dept_stats_df = pd.DataFrame(analysis['department_stats'])
            dept_stats_df.to_excel(writer, sheet_name='По подразделениям', index=False)
            
            # Исходные данные
            analysis['raw_data'].to_excel(writer, sheet_name='Исходные данные', index=False)
            
            # Сводная таблица
            pivot = analysis['raw_data'].pivot_table(
                index='Виновное Подразделение/Клиника',
                columns='is_overdue',
                values='days_to_response',
                aggfunc='count',
                fill_value=0
            )
            pivot.columns = ['В срок', 'Просрочено']
            pivot['Всего'] = pivot['В срок'] + pivot['Просрочено']
            pivot['% просрочки'] = round(pivot['Просрочено'] / pivot['Всего'] * 100, 1)
            pivot.to_excel(writer, sheet_name='Сводная таблица')


if __name__ == "__main__":
    try:
        # Инициализация классов
        data_loader = DataLoader("table1.xlsx")
        data_loader.load_data()
        
        analyzer = ComplaintsAnalyzer(data_loader)
        
        report_generator = ReportGenerator(analyzer)
        
        # Вывод статистики в консоль
        report_generator.print_detailed_stats(30)
        
        # Сохранение отчета в Excel
        report_generator.save_to_excel("medical_complaints_analysis.xlsx")
        
        print("\nАнализ успешно завершен. Результаты сохранены в medical_complaints_analysis.xlsx")
        
    except Exception as e:
        print(f"Ошибка: {e}")
        print("Проверьте путь к файлу и его структуру")


Общая статистика по обращениям:
Порог просрочки: 30 дней
Всего обращений: 101
Просроченных обращений: 0 (0.0%)
Среднее время ответа: -0.0 дней
Максимальное время ответа: 0.0 дней
Минимальное время ответа: -1.0 дней

Статистика по подразделениям (отсортировано по % просрочки):
Подразделение                                                Всего      Просрочено % проср    Доля от общ     Ср.дни     Мин        Макс      
Астрадамский, Азбука Здоровья                                1          0          0.0        nan%            nan        nan        nan       
Благовещенка (вз) (в сети КППн)                              7          0          0.0        nan%            0.0        0.0        0.0       
Брянск                                                       1          0          0.0        nan%            0.0        0.0        0.0       
Бутово +ДА (в сети КППн)                                     1          0          0.0        nan%            0.0        0.0        0.0       
Гамалеи